In [4]:
import matplotlib
matplotlib.use("TkAgg")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def describe_basis(basis):
    return "Z" if basis == 0 else "X"

def describe_bit(bit):
    return str(bit)

def generate_example_data(n=5, p_eve=0.5):
    alice_bits = np.random.randint(2, size=n)
    alice_bases = np.random.randint(2, size=n)
    bob_bases = np.random.randint(2, size=n)
    eavesdrop_events = (np.random.rand(n) < p_eve)

    qubit_data = []
    for i in range(n):
        qubit_data.append({
            "bit": alice_bits[i],
            "alice_basis": alice_bases[i],
            "bob_basis": bob_bases[i],
            "eve": eavesdrop_events[i]
        })
    return qubit_data

class BB84Animator:
    def __init__(self, qubit_data):
        self.qubit_data = qubit_data
        self.n = len(qubit_data)

        self.fig, self.ax = plt.subplots(figsize=(8, 6))
        self.ax.set_xlim(-1, 11)
        self.ax.set_ylim(-1, 6)
        self.ax.axis("off")

        self.pos_alice = (0, 3)
        self.pos_eve   = (5, 3)
        self.pos_bob   = (10, 3)

        self.title_text = self.ax.text(0.5, 1.03, "", transform=self.ax.transAxes,
                                       ha="center", va="bottom", fontsize=14, color="blue")
        self.final_keys_text = self.ax.text(0.5, 0.05, "", transform=self.ax.transAxes,
                                            ha="center", va="center", fontsize=10, color="darkred")

        self.draw_static_scene()

        self.bit_markers = []
        for q in qubit_data:
            circle = plt.Circle((self.pos_alice[0], self.pos_alice[1]), radius=0.3,
                                color="cyan", alpha=0.7)
            self.ax.add_patch(circle)
            label_str = f"Bit={describe_bit(q['bit'])}, A-{describe_basis(q['alice_basis'])}, B-{describe_basis(q['bob_basis'])}"
            label = self.ax.text(self.pos_alice[0], self.pos_alice[1]+0.6,
                                 label_str, ha="center", va="bottom", fontsize=8, color="black")
            self.bit_markers.append({"circle": circle, "label": label})

    def draw_static_scene(self):
        self.ax.text(self.pos_alice[0], self.pos_alice[1]+1, "Alice",
                     ha="center", va="bottom", fontsize=12, color="green")
        self.ax.text(self.pos_eve[0], self.pos_eve[1]+1, "Eve",
                     ha="center", va="bottom", fontsize=12, color="red")
        self.ax.text(self.pos_bob[0], self.pos_bob[1]+1, "Bob",
                     ha="center", va="bottom", fontsize=12, color="blue")

        self.ax.plot(
            [self.pos_alice[0], self.pos_eve[0], self.pos_bob[0]],
            [self.pos_alice[1], self.pos_eve[1], self.pos_bob[1]],
            '--', color="gray"
        )

    def init_anim(self):
        self.title_text.set_text("BB84 Animation Start")
        self.final_keys_text.set_text("")
        for marker in self.bit_markers:
            marker["circle"].set_center((-2, -2))
            marker["label"].set_position((-2, -2))
        return (
            [marker["circle"] for marker in self.bit_markers]
            + [marker["label"] for marker in self.bit_markers]
            + [self.title_text, self.final_keys_text]
        )

    def update_anim(self, frame):
        steps_per_qubit = 60
        qubit_index = frame // steps_per_qubit

        if qubit_index >= self.n:
            self.title_text.set_text("All transmissions complete.")
            self.final_keys_text.set_text(self.compute_final_demo_key())
            return []

        step_in_qubit = frame % steps_per_qubit
        self.title_text.set_text(f"Transmitting Qubit #{qubit_index+1}/{self.n}")

        qubit = self.qubit_data[qubit_index]
        circle = self.bit_markers[qubit_index]["circle"]
        label  = self.bit_markers[qubit_index]["label"]

        # If eavesdropping
        if qubit["eve"]:
            if step_in_qubit <= 20:
                frac = step_in_qubit / 20
                x_pos = self.pos_alice[0] + frac*(self.pos_eve[0] - self.pos_alice[0])
                y_pos = self.pos_alice[1]
            elif step_in_qubit <= 40:
                x_pos = self.pos_eve[0]
                y_pos = self.pos_eve[1]
            else:
                frac = (step_in_qubit - 40) / 20
                x_pos = self.pos_eve[0] + frac*(self.pos_bob[0] - self.pos_eve[0])
                y_pos = self.pos_eve[1]
        else:
            if step_in_qubit <= 40:
                frac = step_in_qubit / 40
                x_pos = self.pos_alice[0] + frac*(self.pos_bob[0] - self.pos_alice[0])
                y_pos = self.pos_alice[1]
            else:
                x_pos = self.pos_bob[0]
                y_pos = self.pos_bob[1]

        circle.set_center((x_pos, y_pos))
        label.set_position((x_pos, y_pos + 0.6))

        # Upon arrival at Bob
        if x_pos == self.pos_bob[0]:
            if qubit["eve"]:
                circle.set_color("orange")
            else:
                circle.set_color("cyan")

            if qubit["alice_basis"] != qubit["bob_basis"]:
                circle.set_alpha(0.3)
            else:
                circle.set_alpha(1.0)
        return []

    def compute_final_demo_key(self):
        kept_indices = []
        for i, q in enumerate(self.qubit_data):
            if q["alice_basis"] == q["bob_basis"]:
                kept_indices.append(i)
        if not kept_indices:
            return "No qubits matched bases."
        lines = [
            f"Kept qubits (same basis): {', '.join(str(i+1) for i in kept_indices)}",
            "Final key length would be reduced by sampling & error check."
        ]
        return "\n".join(lines)

def animate_bb84(n=5, p_eve=0.5):
    qubit_data = generate_example_data(n=n, p_eve=p_eve)
    animator = BB84Animator(qubit_data)

    anim = FuncAnimation(
        animator.fig,
        animator.update_anim,
        init_func=animator.init_anim,
        frames=1000,
        interval=50,
        blit=False,
        repeat=False
    )
    
    plt.show(block=True)
    return anim

if __name__ == "__main__":
    my_anim = animate_bb84(n=50, p_eve=0.5)